# Data Mining for Business Analytics

## Model Evaluation Measures - Pt. 2

Spring 2019 - Prof. George Valkanas

Material based on content courtesy of Prof. Foster Provost

***

In [ ]:
# Import the libraries we will be using
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8

### Confusion Matrix

Just as a reminder, a confusion matrix looks like this

<br/>
<table width="60%">
    <tbody>
        <tr style="background: rgba(255, 255, 255, 0.1)">
            <td colspan="2" width="30%"></td>
            <td colspan="2" style="border: 2px solid black; background: white; text-align: center" ><b>True Class</b></td>
        </tr>
        <tr style="background: rgba(255, 255, 255, 0.1)">
            <td colspan="2"></td>
            <td style="border: 2px solid black; background: #E8E8E8; text-align: center"><b>Positive (1)</b></td>
            <td style="border: 2px solid black; background: #E8E8E8; text-align: center"><b>Negative (0)</b></td>
        </tr>
        <tr>
            <td rowspan="2" style="border: 2px solid black; background: white; text-align: center"><b>Predicted<br/>Class</b></td>
            <td style="border: 2px solid black; background: #E8E8E8; text-align: center"><b>Positive<br/>("Yes")</b></td>
            <td style="border: 2px solid black; background: lightgreen; text-align: center">
                <b>True Positive (TP)</b>
            </td>
            <td style="border: 2px solid black; background: #ff9999; text-align: center">
                <b>False Positive (FP)</b>
            </td>
        </tr>
        <tr>
            <td style="border: 2px solid black; background: #E8E8E8; text-align: center; text-align: center">
                <b>Negative<br/>("No")</b>
            </td>
            <td style="background: #ff9999; border: 2px solid black; text-align: center">
                <b>False Negative (FN)</b>
            </td>
            <td style="background: lightgreen; border: 2px solid black; text-align: center">
                <b>True Negative (TN)</b>
            </td>
        </tr>
    </tbody>
</table>


We've already seen the _Cumulative Response Curve_ (CRC) and the process that we use to generate it. There are several other curves that we can use to evaluate a model. Some of them are quite similar to the CRC, whereas others differ more.
***

### Some Additional Measures

To produce the extra curves, we must first introduce some additional measures, which we can build from our confusion matrix.

* **True Positive Rate (TPR):** Also known as <u>_recall_</u> is the portion of actually true instances ("truely" to the positive class) that were classified / captured / identified as such. To compute TPR, we use the "Positive (1)" column of our confusion matrix. Formally:

$$ TPR = \frac{TP}{TP + FN} $$

<br/>

* **False Positive Rate (FPR):** Roughly speaking, how likely we are to _misclassify_ a "truely" negative instance as a positive one. To compute FPR, we use the "Negative (0)" column of our confusion matrix. Formally:

$$ FPR = \frac{FP}{FP + TN}$$


**VERY IMPORTANT**: As you can see from the previous quantities, TPR and FPR use values from our _confusion matrix_. What does that mean, with respect to a classifier? Recall that our models are _scoring models_ !
***

### The ROC  


_Not to be confused with [The Rock](https://www.imdb.com/name/nm0425005/)!_


The Receiver Operating Characteristic (ROC) Curve uses the two measures that we introduced above: **FPR** and **TPR**.
It helps us to visualize the _trade-offs_ between the **opportunity for benefits** (via true positives on the y-axis) and the **possibility of costs** (via false positives on the x-axis).

The ROC _also_ expects our predictions to be _ordered_ according to their probability estimates. Let's briefly examine how it works.

<img src="images/ROC1.png" alt="Drawing" style="width: 600px;"/>


The four corners of an ROC curve:

* **(0, 0):** Represents the strategy of never issuing a positive classification
* **(0, 1):** Represents **perfect classification**, i.e. everything is classified to the class it really belongs! 
* **(1, 0):** We somehow managed to get _everything_ wrong!?
* **(1, 1):** Always issue positive classification (example: "everyone's pregnant!")

The diagonal line between **(0, 0)** and **(1, 1)** represents a randomly guessing "classifier" (e.g., flipping a weighted coin).

- _Provost, Foster, and Tom Fawcett. Data Science for Business: _
  _What you need to know about data mining and data-analytic thinking. O'Reilly Media, Inc., 2013._


<img src="images/ROC2.png" alt="Drawing" style="width: 600px;"/>

***

### ROC AUC

Finally, we can measure the _Area Under the (ROC) Curve_ (ROC AUC), which is the fraction of the total area below our classifier's ROC curve.
_Remember:_ Despite its simplicity, you should approach the AUC just like any other evaluation measures: whether it's the right thing to use depends on your business application and domain!

Let's go back to our mailing campaign example and compute the AUC there!

In [ ]:
original = pd.read_csv("data/mailing.csv")  # Load the data
original.head()  # Let's take a look at the data

In [ ]:
data = original.copy()
for field in ['rfaa2', 'pepstrfl']:  # Do the same thing for the two fields of interest
    # Go through each possible value 
    for value in data[field].unique():
        # Create a new binary field
        data[field + "_" + value] = pd.Series(data[field] == value, dtype=int)

    # Drop the original field
    data = data.drop([field], axis=1)
    
# Let's look at the data again, after the modifications
data.head()

In [ ]:
# The "data" dataframe contains everything together.
# Let's be picky this time w.r.t. the features that we'll use
data['date_diff'] = 1 + (data['Lastdate'] - data['Firstdate'])
X = data[['Income', 'Amount', 'rfaf2', 'glast', 'gavr', 'date_diff']]
Y = data['class']

# Keep 75% of the data for training and 25% for testing. We have used this method before.
X_mailing_train, X_mailing_test, Y_mailing_train, Y_mailing_test = train_test_split(X, Y, train_size=.75)

In [ ]:

# Pick a list of regularization values
regul_vals = [0.001, 0.01, 1.0]


for c in regul_vals:

    # Fit a logistic regression model with the respective parameter.
    model = LogisticRegression(C=c)
    model.fit(X_mailing_train, Y_mailing_train)

    # Get the probability of Y_test records being = 1
    Y_test_probability_1 = model.predict_proba(X_mailing_test)[:, 1]

    # Use the metrics.roc_curve function to get the true positive rate (tpr) and false positive rate (fpr)
    # We will not use the thresholds parameter, but it is returned by the method
    fpr, tpr, thresholds = metrics.roc_curve(Y_mailing_test, Y_test_probability_1)

    # Compute the AUC for the ROC
    auc = metrics.roc_auc_score(Y_mailing_test, Y_test_probability_1)

    # Plot the ROC curve. Report the AUC in the legend
    plt.plot(fpr, tpr, label="AUC (C=%.3f) = %.2f" % (c, round(auc, 2)))
    
plt.xlabel("False positive rate (fpr)")
plt.ylabel("True positive rate (tpr)")
plt.plot([0,1], [0,1], 'k--', label="Random")
plt.legend(loc=2)

### The Lift Curve

The ROC Curve has been a fundamental tool in evaluating the performance of classifiers, for a number of reasons. (can you think of any?).

A potential downside is that **FPR** and **TPR** may be tricky quantities to express / explain in a business setting. The CRC is perhaps more "approachable".

Another type of curve that we often user is known as the **Lift Curve**. It tells us _how much better_ is one model with respect to another. To compute our Lift Curve, we will start with our CRC.

In [ ]:

# Let's switch gears again and see what happens if we use all the features
X = data.drop(['class'], axis=1)

# Keep 75% of the data for training and 25% for testing. We have used this method before.
X_mailing_train, X_mailing_test, Y_mailing_train, Y_mailing_test = train_test_split(X, Y, train_size=.75)

In [ ]:
def train_and_compute_crc( model, x_train, y_train, x_test, y_test):

    # Train the model
    model.fit(x_train, y_train)

    # Let's get the probabilities once more
    probabilities = model.predict_proba(x_test)[:, 1]

    ## We practically ask for descending order of the probabilities
    order = np.argsort(probabilities)[::-1]

    actual_class_sorted = np.array(y_test)[order]  # 
    return np.cumsum(actual_class_sorted)


In [ ]:
# Let's train a few classifiers here and try to plot everything together.
model = LogisticRegression(C=10)
logReg10_crc = train_and_compute_crc( model, X_mailing_train, Y_mailing_train, X_mailing_test, Y_mailing_test )

model = DecisionTreeClassifier(criterion='entropy', max_depth=15)
dec_tree_crc = train_and_compute_crc( model, X_mailing_train, Y_mailing_train, X_mailing_test, Y_mailing_test )

In [ ]:
# For the x-axis compute the fraction of instances we have seen so far
x_lift = np.cumsum( np.ones(len(X_mailing_test)) )
x_lift = x_lift / len(X_mailing_test)


# Compute the LIFT for the decision tree.
# a) Normalize the CRC by the maximum value
# b) Divide each value by the fraction of items we have seen so far
dec_tree_lift = dec_tree_crc / np.max(dec_tree_crc)
dec_tree_lift = dec_tree_lift / x_lift


# Compute the LIFT for the Logistic Regression.
# a) Normalize the CRC by the maximum value
# b) Divide each value by the fraction of items we have seen so far
log_reg_lift = logReg10_crc / np.max(logReg10_crc)
log_reg_lift = log_reg_lift / x_lift

In [ ]:

# Let's plot the information

plt.figure(figsize=(10,7))  # Create a big enough plot

plt.plot(x_lift * 100, dec_tree_lift, label="Decision Tree")  # Show the LIFT curve for the Decision Tree

plt.plot(x_lift * 100, log_reg_lift, label="Logistic Regression")  # Show the LIFT curve for Logistic Regression

plt.plot([0,100], [1,1], 'k--', label="Random")
plt.xlabel("Percentage of test instances (decreasing score)")
plt.ylabel("Lift (times)")
plt.title("Lift curve")
plt.legend()
plt.show()

In [ ]:

# Let's plot the information

plt.figure(figsize=(10,7))  # Create a big enough plot

plt.plot(x_lift * 100, dec_tree_lift, label="Decision Tree")  # Show the LIFT curve for the Decision Tree

plt.plot(x_lift * 100, log_reg_lift, label="Logistic Regression")  # Show the LIFT curve for Logistic Regression

plt.plot([0,100], [1,1], 'k--', label="Random")
plt.xlim(-0.3, 5)
plt.xlabel("Percentage of test instances (decreasing score)")
plt.ylabel("Lift (times)")
plt.title("Lift curve")
plt.legend()
plt.show()

**Question:** What can you tell me by looking at the above graph?
***

### Profit Curves

Finally, you should always remember that there is _evaluating the model_ and _evaluating the business_. The two may be linked in one way or another, but this connection has probably not been clear so far. So let's make this happen!

Let's generate a simple matrix like the one below.
**BE CAREFUL**: The matrix has the structure of a confusion matrix, _but it is not one!_

**Question:** Can you see why?

In [ ]:
cost_matrix = pd.DataFrame([[17, -1], [0, 0]], columns=['(True) p', '(True) n'], index=['[Predicted] Y', '[Predicted] N'])
cost_matrix

The above matrix is a _profit matrix_. It says what we _gain_ by targeting a customer that the classifier said that we should target (predicted = "Y").

We will basically combine our profit matrix with _each_ of our confusion matrices, i.e., for each threshold.

In [ ]:
model = DecisionTreeClassifier(criterion='entropy', max_depth=15)
model.fit(X_mailing_train, Y_mailing_train)
model_probas = model.predict(X_mailing_test) # [:, 1]


# Get the false positive rate, true positive rate, and all thresholds
fpr, tpr, thresholds = metrics.roc_curve(Y_mailing_test, model_probas)

Size_targeted_pop = float(len(Y_mailing_test))

# What is the baseline probability of being positive or negative in the data set?
p_p = np.sum(Y_mailing_test)/float(len(Y_mailing_test))
p_n = 1 - np.sum(Y_mailing_test)/float(len(Y_mailing_test))


# How many users are above the current threshold?
n_targeted = []
for t in thresholds:
    n_targeted.append(np.sum(model_probas >= t))


# Turn these counts to percentages of users above the threshold
n_targeted = np.array(n_targeted)/float(len(Y_mailing_test))

# Expected profits:  
expected_profits = (cost_matrix['(True) p']['[Predicted] Y']*(tpr*p_p)) + (cost_matrix['(True) n']['[Predicted] Y']*(fpr*p_n))

# Plot the profit curve
plt.plot(n_targeted, Size_targeted_pop*expected_profits)
plt.xlabel("Percentage of users targeted")
plt.ylabel("Profit")
plt.title("Profits")
plt.show()


# # Let's compute the confusion matrix first.
# # To do that we need the actual classes and the predicted ones
# conf_mtx = metrics.confusion_matrix(Y_mailing_test, predictions, labels=[1, 0])

# # Let's turn the confusion matrix into a DataFrame, to make it more presentable
# conf_mtx_df = pd.DataFrame(conf_mtx.T, columns=[, ], index=[, '[Predicted] N'])
# conf_mtx_df